<a href="https://colab.research.google.com/github/niapollard/Benchmarking_QC/blob/main/Varying%20Basis%20Set%20Al3_5%20trials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q jarvis-tools qiskit==0.43.1

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 13.4 MB/s eta 0:00:00
   

In [2]:
!pip install qiskit_nature==0.6.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 23.8 MB/s eta 0:00:00


In [3]:
!pip install pyscf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 MB 10.7 MB/s eta 0:00:00


In [4]:
!pip install qiskit-aer==0.12.0

In [5]:
import numpy as np

# Importing Qiskit and Jarvis libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit import Aer
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit_nature.drivers.second_quantization import PySCFDriver
from qiskit_nature.drivers.second_quantization import MethodType
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit_nature.transformers.second_quantization.electronic.active_space_transformer import ActiveSpaceTransformer
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SLSQP, COBYLA, L_BFGS_B, SPSA
import itertools, functools
from qiskit.opflow import I, X, Y, Z
from jarvis.db.figshare import get_wann_electron, get_wann_phonon, get_hk_tb
from jarvis.core.circuits import QuantumCircuitLibrary
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
from pyscf import gto, scf, tools

<ipython-input-5-fba5fa152d80>:29: NatureDeprecationWarning: The qiskit_nature.algorithms.excited_state_solvers package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the qiskit_nature.second_q.algorithms.excited_state_solvers package.
  from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
<ipython-input-5-fba5fa152d80>:29: NatureDeprecationWarning: The qiskit_nature.algorithms.pes_sampler package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release . Please refer to 'https://github.com/Qiskit/qiskit-nature/issues/750' for additional information regarding 'extrapolators'.
  from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver


In [6]:
#Build the molecule
molecule = Molecule(
    geometry=[['Al',[-3.12653169, 0.21001977, 0.00000000]],
              ['Al',[-0.60201643, 0.29584456, 0.00000000]],
              ['Al',[-1.93850704, 2.43904280, 0.00000000]]],
    charge=-1,
    multiplicity=1)

# molecule = Molecule(
#     geometry=[['Al',[0,0, 0.00000000]]],
#     charge=-1,
#     multiplicity=1)

In [7]:
import time

start_time = time.time()

In [8]:
def get_qubit_op(
    molecule,
    basis,
    functional,
    method,
    driver_type= ElectronicStructureDriverType.PYSCF,
    mapper= JordanWignerMapper()):

  driver=ElectronicStructureMoleculeDriver(molecule, basis, method,driver_type)
  properties = driver.run()
  problem = ElectronicStructureProblem(driver)

  second_q_ops = problem.second_q_ops()
  second_q_ops = list(second_q_ops.values()) #list of second quantized operators
  hamiltonian=second_q_ops[0]

  numpy_solver = NumPyMinimumEigensolver()
  tmp = properties.get_property('ParticleNumber')
  alpha_occ=tmp.occupation_alpha
  beta_occ=tmp.occupation_beta

  mo_considered=3
  #active and inactive space has to be even, non-magnetic

  first_index = min(np.where(alpha_occ<1)[0][0],np.where(beta_occ<1)[0][0])

  orb_act = np.arange(first_index-mo_considered,first_index)+1

  transformer= ActiveSpaceTransformer(num_electrons=mo_considered+1, num_molecular_orbitals=len(orb_act), active_orbitals=orb_act)
  problem_reduced = ElectronicStructureProblem(driver, [transformer])
  second_q_ops_reduced = problem_reduced.second_q_ops()

  second_q_ops_reduced = list(second_q_ops_reduced.values())
  hamiltonian_reduced = second_q_ops_reduced[0]

  # mapper = JordanWignerMapper()
  converter = QubitConverter(mapper=mapper)
  qubit_op = converter.convert(hamiltonian_reduced)

  res1={}
  res1['qubit_op']=qubit_op
  res1['converter']=converter
  res1['problem_reduced']=problem_reduced
  res1['numpy_solver']=numpy_solver
  return res1 #qubit_op, converter, problem_reduced

def get_energy(optimizer='',device='',qubit_op='',seed='42', reps=1):
  #seed = 42
  counts = []
  values = []


  def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)

  algorithm_globals.random_seed = seed
  np.random.seed(seed)
  print('device',device)
  print('seed',seed)

  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
  n_qubits = qubit_op.num_qubits
  ansatz = QuantumCircuitLibrary(n_qubits=n_qubits, reps=reps).circuit6()
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)
  result = vqe.compute_minimum_eigenvalue(operator=qubit_op)


  eigenvalue = result.eigenvalue
  # return eigenvalue, vqe, qi

  res={}
  res['eigenvalue']=eigenvalue
  res['vqe']=vqe
  res['qi']=qi
  return res

elements=['Al']
basis=['631g','sto3g', '321g', 'sto6g', '631g','lanl2dz']
method=[MethodType.RKS]
#driver_type=[]
optimizer=[SLSQP(maxiter=1000)]
mapper=[JordanWignerMapper()]
functionals=['lda']
devices=[Aer.get_backend('statevector_simulator')]
reps_list = [1]
mem={}
for i in elements:
  for j in basis:
    for k in method:
      #for l in driver_type:
        for m in optimizer:
          for n in mapper:
            for o in functionals:
              for p in devices:
                for reps in reps_list:
                  print(i,j,k,m,n,o,p,reps)

                  res1=get_qubit_op(molecule=molecule,basis=j,mapper=n,functional=o,method=k)
                  res=get_energy(optimizer=m,device=p,qubit_op=res1['qubit_op'],seed=42)
                  print(res['eigenvalue'])

                  #GroundStateEigensolver
                  print("VQE Energy Result")
                  solver =GroundStateEigensolver(res1['converter'], res['vqe'])
                  result=solver.solve(res1['problem_reduced'])
                  print(result)

                  #numpy
                  print("Numpy Energy Result")
                  solver =GroundStateEigensolver(res1['converter'], res1['numpy_solver'])
                  result=solver.solve(res1['problem_reduced'])
                  print(result)

                  mem[i,j,k,m,n,o,p,reps]= {
                      'eigenvalue': res['eigenvalue'],
                      'vqe': res['vqe'],
                      'qi': res['qi'],
                      'converter': res1['converter'],
                      'problem_reduced': res1['problem_reduced']
                     }

Al 631g MethodType.RKS <qiskit.algorithms.optimizers.slsqp.SLSQP object at 0x7e9e570d75b0> <qiskit_nature.mappers.second_quantization.jordan_wigner_mapper.JordanWignerMapper object at 0x7e9e605d8d00> lda statevector_simulator 1


<ipython-input-8-eb8e343ad184>:99: NatureDeprecationWarning: The qiskit_nature.problems.second_quantization.electronic.ElectronicStructureProblem class is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the qiskit_nature.second_q.problems.ElectronicStructureProblem class.
  res1=get_qubit_op(molecule=molecule,basis=j,mapper=n,functional=o,method=k)
<ipython-input-8-eb8e343ad184>:17: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  numpy_solver = NumPyMinimumEigensolver()


device statevector_simulator
seed 42


<ipython-input-8-eb8e343ad184>:62: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
<ipython-input-8-eb8e343ad184>:65: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)


(8.5066017987e-08+0j)
VQE Energy Result


...

(8.5066017987e-08+0j)
VQE Energy Result
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -831.820030951415
  - computed part:      -1.371347559814
  - ActiveSpaceTransformer extracted energy part: -830.448683391601
~ Nuclear repulsion energy (Hartree): 106.218168150552
> Total ground state energy (Hartree): -725.601862800864
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.511 S^2: 0.771 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [-139.21936841  72.34588319  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [-142.79164006  74.1497234  -0.00023779]
    - computed part:      [-14.28145996  7.36857871  -0.00023779]
    - ActiveSpaceTransformer extracted energy part: [-128.5101801  66.78114469  0.0]
  > Dipole moment (a.u.): [3.57227165  -1.80384021  0.00023779]  Total: 4.0018701
                 (debye): [9.079808  -4.58490406  0.00060441]  Total: 10.17173825
 
Numpy Energy Result
=== GROUND STATE ENERGY ===
 
* Electronic gr

<ipython-input-8-eb8e343ad184>:17: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  numpy_solver = NumPyMinimumEigensolver()


device statevector_simulator
seed 42


<ipython-input-8-eb8e343ad184>:62: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
<ipython-input-8-eb8e343ad184>:65: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)


(8.5066017987e-08+0j)
VQE Energy Result
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -247.938891926001
  - computed part:      -1.240115742037
  - ActiveSpaceTransformer extracted energy part: -246.698776183964
~ Nuclear repulsion energy (Hartree): 106.218168150552
> Total ground state energy (Hartree): -141.72072377545
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.369 S^2: 0.504 M: -0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [-139.21936841  72.34588319  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [-137.47168758  77.84898293  0.00911928]
    - computed part:      [-9.39155037  10.93771565  -0.01861597]
    - ActiveSpaceTransformer extracted energy part: [-128.08013722  66.91126727  0.02773525]
  > Dipole moment (a.u.): [-1.74768083  -5.50309974  -0.00911928]  Total: 5.77395689
                 (debye): [-4.44216115  -13.98748301  -0.0231789]  Total: 14.67593315
 
Numpy Energy Result
=== GROUND STATE ENERGY ===
 


In [9]:
# Record the end time
end_time = time.time()

# Calculate the duration
duration = end_time - start_time

# Print the duration
print("Job took {:.2f} seconds to run.".format(duration))

Job took 368.71 seconds to run.


In [10]:
start_time = time.time()

In [11]:
def get_qubit_op(
    molecule,
    basis,
    functional,
    method,
    driver_type= ElectronicStructureDriverType.PYSCF,
    mapper= JordanWignerMapper()):

  driver=ElectronicStructureMoleculeDriver(molecule, basis, method,driver_type)
  properties = driver.run()
  problem = ElectronicStructureProblem(driver)

  second_q_ops = problem.second_q_ops()
  second_q_ops = list(second_q_ops.values()) #list of second quantized operators
  hamiltonian=second_q_ops[0]

  numpy_solver = NumPyMinimumEigensolver()
  tmp = properties.get_property('ParticleNumber')
  alpha_occ=tmp.occupation_alpha
  beta_occ=tmp.occupation_beta

  mo_considered=3
  #active and inactive space has to be even, non-magnetic

  first_index = min(np.where(alpha_occ<1)[0][0],np.where(beta_occ<1)[0][0])

  orb_act = np.arange(first_index-mo_considered,first_index)+1

  transformer= ActiveSpaceTransformer(num_electrons=mo_considered+1, num_molecular_orbitals=len(orb_act), active_orbitals=orb_act)
  problem_reduced = ElectronicStructureProblem(driver, [transformer])
  second_q_ops_reduced = problem_reduced.second_q_ops()

  second_q_ops_reduced = list(second_q_ops_reduced.values())
  hamiltonian_reduced = second_q_ops_reduced[0]

  # mapper = JordanWignerMapper()
  converter = QubitConverter(mapper=mapper)
  qubit_op = converter.convert(hamiltonian_reduced)

  res1={}
  res1['qubit_op']=qubit_op
  res1['converter']=converter
  res1['problem_reduced']=problem_reduced
  res1['numpy_solver']=numpy_solver
  return res1 #qubit_op, converter, problem_reduced

def get_energy(optimizer='',device='',qubit_op='',seed='42', reps=1):
  #seed = 42
  counts = []
  values = []


  def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)

  algorithm_globals.random_seed = seed
  np.random.seed(seed)
  print('device',device)
  print('seed',seed)

  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
  n_qubits = qubit_op.num_qubits
  ansatz = QuantumCircuitLibrary(n_qubits=n_qubits, reps=reps).circuit6()
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)
  result = vqe.compute_minimum_eigenvalue(operator=qubit_op)


  eigenvalue = result.eigenvalue
  # return eigenvalue, vqe, qi

  res={}
  res['eigenvalue']=eigenvalue
  res['vqe']=vqe
  res['qi']=qi
  return res

elements=['Al']
basis=['631g','sto3g', '321g', 'sto6g', '631g','lanl2dz']
method=[MethodType.RKS]
#driver_type=[]
optimizer=[SLSQP(maxiter=1000)]
mapper=[JordanWignerMapper()]
functionals=['lda']
devices=[Aer.get_backend('statevector_simulator')]
reps_list = [1]
mem={}
for i in elements:
  for j in basis:
    for k in method:
      #for l in driver_type:
        for m in optimizer:
          for n in mapper:
            for o in functionals:
              for p in devices:
                for reps in reps_list:
                  print(i,j,k,m,n,o,p,reps)

                  res1=get_qubit_op(molecule=molecule,basis=j,mapper=n,functional=o,method=k)
                  res=get_energy(optimizer=m,device=p,qubit_op=res1['qubit_op'],seed=42)
                  print(res['eigenvalue'])

                  #GroundStateEigensolver
                  print("VQE Energy Result")
                  solver =GroundStateEigensolver(res1['converter'], res['vqe'])
                  result=solver.solve(res1['problem_reduced'])
                  print(result)

                  #numpy
                  print("Numpy Energy Result")
                  solver =GroundStateEigensolver(res1['converter'], res1['numpy_solver'])
                  result=solver.solve(res1['problem_reduced'])
                  print(result)

                  mem[i,j,k,m,n,o,p,reps]= {
                      'eigenvalue': res['eigenvalue'],
                      'vqe': res['vqe'],
                      'qi': res['qi'],
                      'converter': res1['converter'],
                      'problem_reduced': res1['problem_reduced']
                     }

Al 631g MethodType.RKS <qiskit.algorithms.optimizers.slsqp.SLSQP object at 0x7e9e542980a0> <qiskit_nature.mappers.second_quantization.jordan_wigner_mapper.JordanWignerMapper object at 0x7e9e54299270> lda statevector_simulator 1


<ipython-input-11-eb8e343ad184>:17: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  numpy_solver = NumPyMinimumEigensolver()


device statevector_simulator
seed 42


<ipython-input-11-eb8e343ad184>:62: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
<ipython-input-11-eb8e343ad184>:65: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)


(8.5066017987e-08+0j)
VQE Energy Result
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -831.82003091059
  - computed part:      -1.371347518989
  - ActiveSpaceTransformer extracted energy part: -830.448683391601
~ Nuclear repulsion energy (Hartree): 106.218168150552
> Total ground state energy (Hartree): -725.601862760039
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.511 S^2: 0.771 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [-139.21936841  72.34588319  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [-142.79168596  74.14970798  -0.00024821]
    - computed part:      [-14.28150586  7.36856329  -0.00024821]
    - ActiveSpaceTransformer extracted energy part: [-128.5101801  66.78114469  0.0]
  > Dipole moment (a.u.): [3.57231755  -1.80382479  0.00024821]  Total: 4.00190412
                 (debye): [9.07992466  -4.58486486  0.0006309]  Total: 10.17182472
 
Numpy Energy Result
=== GROUND STATE ENERGY ===
 
* Electronic 

...

(8.5066017987e-08+0j)
VQE Energy Result
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -831.820031609566
  - computed part:      -1.371348217964
  - ActiveSpaceTransformer extracted energy part: -830.448683391601
~ Nuclear repulsion energy (Hartree): 106.218168150552
> Total ground state energy (Hartree): -725.601863459014
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.511 S^2: 0.771 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [-139.21936841  72.34588319  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [-142.7912319  74.1498657  -0.00001837]
    - computed part:      [-14.2810518  7.36872101  -0.00001837]
    - ActiveSpaceTransformer extracted energy part: [-128.5101801  66.78114469  0.0]
  > Dipole moment (a.u.): [3.57186349  -1.80398251  0.00001837]  Total: 4.0015699
                 (debye): [9.07877056  -4.58526575  0.00004669]  Total: 10.17097522
 
Numpy Energy Result
=== GROUND STATE ENERGY ===
 
* Electronic gr

<ipython-input-11-eb8e343ad184>:17: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  numpy_solver = NumPyMinimumEigensolver()


device statevector_simulator
seed 42


<ipython-input-11-eb8e343ad184>:62: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
<ipython-input-11-eb8e343ad184>:65: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)


(8.5066017987e-08+0j)
VQE Energy Result
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -247.937816119919
  - computed part:      -1.239101574264
  - ActiveSpaceTransformer extracted energy part: -246.698714545656
~ Nuclear repulsion energy (Hartree): 106.218168150552
> Total ground state energy (Hartree): -141.719647969368
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.360 S^2: 0.490 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [-139.21936841  72.34588319  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [-137.31861525  77.72402858  -0.01045773]
    - computed part:      [-9.30724909  10.85746579  -0.01581419]
    - ActiveSpaceTransformer extracted energy part: [-128.01136617  66.86656279  0.00535646]
  > Dipole moment (a.u.): [-1.90075316  -5.37814539  0.01045773]  Total: 5.70415811
                 (debye): [-4.83123217  -13.66988077  0.0265809]  Total: 14.49852238
 
Numpy Energy Result
=== GROUND STATE ENERGY ===
 
*

In [12]:
# Record the end time
end_time = time.time()

# Calculate the duration
duration = end_time - start_time

# Print the duration
print("Job took {:.2f} seconds to run.".format(duration))

Job took 367.24 seconds to run.


In [13]:
start_time = time.time()

In [14]:
def get_qubit_op(
    molecule,
    basis,
    functional,
    method,
    driver_type= ElectronicStructureDriverType.PYSCF,
    mapper= JordanWignerMapper()):

  driver=ElectronicStructureMoleculeDriver(molecule, basis, method,driver_type)
  properties = driver.run()
  problem = ElectronicStructureProblem(driver)

  second_q_ops = problem.second_q_ops()
  second_q_ops = list(second_q_ops.values()) #list of second quantized operators
  hamiltonian=second_q_ops[0]

  numpy_solver = NumPyMinimumEigensolver()
  tmp = properties.get_property('ParticleNumber')
  alpha_occ=tmp.occupation_alpha
  beta_occ=tmp.occupation_beta

  mo_considered=3
  #active and inactive space has to be even, non-magnetic

  first_index = min(np.where(alpha_occ<1)[0][0],np.where(beta_occ<1)[0][0])

  orb_act = np.arange(first_index-mo_considered,first_index)+1

  transformer= ActiveSpaceTransformer(num_electrons=mo_considered+1, num_molecular_orbitals=len(orb_act), active_orbitals=orb_act)
  problem_reduced = ElectronicStructureProblem(driver, [transformer])
  second_q_ops_reduced = problem_reduced.second_q_ops()

  second_q_ops_reduced = list(second_q_ops_reduced.values())
  hamiltonian_reduced = second_q_ops_reduced[0]

  # mapper = JordanWignerMapper()
  converter = QubitConverter(mapper=mapper)
  qubit_op = converter.convert(hamiltonian_reduced)

  res1={}
  res1['qubit_op']=qubit_op
  res1['converter']=converter
  res1['problem_reduced']=problem_reduced
  res1['numpy_solver']=numpy_solver
  return res1 #qubit_op, converter, problem_reduced

def get_energy(optimizer='',device='',qubit_op='',seed='42', reps=1):
  #seed = 42
  counts = []
  values = []


  def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)

  algorithm_globals.random_seed = seed
  np.random.seed(seed)
  print('device',device)
  print('seed',seed)

  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
  n_qubits = qubit_op.num_qubits
  ansatz = QuantumCircuitLibrary(n_qubits=n_qubits, reps=reps).circuit6()
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)
  result = vqe.compute_minimum_eigenvalue(operator=qubit_op)


  eigenvalue = result.eigenvalue
  # return eigenvalue, vqe, qi

  res={}
  res['eigenvalue']=eigenvalue
  res['vqe']=vqe
  res['qi']=qi
  return res

elements=['Al']
basis=['631g','sto3g', '321g', 'sto6g', '631g','lanl2dz']
method=[MethodType.RKS]
#driver_type=[]
optimizer=[SLSQP(maxiter=1000)]
mapper=[JordanWignerMapper()]
functionals=['lda']
devices=[Aer.get_backend('statevector_simulator')]
reps_list = [1]
mem={}
for i in elements:
  for j in basis:
    for k in method:
      #for l in driver_type:
        for m in optimizer:
          for n in mapper:
            for o in functionals:
              for p in devices:
                for reps in reps_list:
                  print(i,j,k,m,n,o,p,reps)

                  res1=get_qubit_op(molecule=molecule,basis=j,mapper=n,functional=o,method=k)
                  res=get_energy(optimizer=m,device=p,qubit_op=res1['qubit_op'],seed=42)
                  print(res['eigenvalue'])

                  #GroundStateEigensolver
                  print("VQE Energy Result")
                  solver =GroundStateEigensolver(res1['converter'], res['vqe'])
                  result=solver.solve(res1['problem_reduced'])
                  print(result)

                  #numpy
                  print("Numpy Energy Result")
                  solver =GroundStateEigensolver(res1['converter'], res1['numpy_solver'])
                  result=solver.solve(res1['problem_reduced'])
                  print(result)

                  mem[i,j,k,m,n,o,p,reps]= {
                      'eigenvalue': res['eigenvalue'],
                      'vqe': res['vqe'],
                      'qi': res['qi'],
                      'converter': res1['converter'],
                      'problem_reduced': res1['problem_reduced']
                     }

Al 631g MethodType.RKS <qiskit.algorithms.optimizers.slsqp.SLSQP object at 0x7e9e54299810> <qiskit_nature.mappers.second_quantization.jordan_wigner_mapper.JordanWignerMapper object at 0x7e9e5429aa70> lda statevector_simulator 1


<ipython-input-14-eb8e343ad184>:17: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  numpy_solver = NumPyMinimumEigensolver()


device statevector_simulator
seed 42


<ipython-input-14-eb8e343ad184>:62: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
<ipython-input-14-eb8e343ad184>:65: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)


(8.5066017987e-08+0j)
VQE Energy Result
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -831.820031614239
  - computed part:      -1.371348222638
  - ActiveSpaceTransformer extracted energy part: -830.448683391601
~ Nuclear repulsion energy (Hartree): 106.218168150552
> Total ground state energy (Hartree): -725.601863463687
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.511 S^2: 0.771 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [-139.21936841  72.34588319  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [-142.79136301  74.14983453  -0.00002307]
    - computed part:      [-14.28118291  7.36868984  -0.00002307]
    - ActiveSpaceTransformer extracted energy part: [-128.5101801  66.78114469  0.0]
  > Dipole moment (a.u.): [3.5719946  -1.80395134  0.00002307]  Total: 4.00167288
                 (debye): [9.07910381  -4.58518651  0.00005863]  Total: 10.17123696
 
Numpy Energy Result
=== GROUND STATE ENERGY ===
 
* Electronic

...

(8.5066017987e-08+0j)
VQE Energy Result
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -831.820030932327
  - computed part:      -1.371347540726
  - ActiveSpaceTransformer extracted energy part: -830.448683391601
~ Nuclear repulsion energy (Hartree): 106.218168150552
> Total ground state energy (Hartree): -725.601862781775
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.511 S^2: 0.772 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [-139.21936841  72.34588319  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [-142.79164438  74.1497164  -0.00024658]
    - computed part:      [-14.28146428  7.36857171  -0.00024658]
    - ActiveSpaceTransformer extracted energy part: [-128.5101801  66.78114469  0.0]
  > Dipole moment (a.u.): [3.57227597  -1.80383321  0.00024658]  Total: 4.0018708
                 (debye): [9.07981899  -4.58488626  0.00062674]  Total: 10.17174003
 
Numpy Energy Result
=== GROUND STATE ENERGY ===
 
* Electronic 

<ipython-input-14-eb8e343ad184>:17: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  numpy_solver = NumPyMinimumEigensolver()


device statevector_simulator
seed 42


<ipython-input-14-eb8e343ad184>:62: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
<ipython-input-14-eb8e343ad184>:65: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)


(8.5066017987e-08+0j)
VQE Energy Result
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -248.007026437908
  - computed part:      -1.303106791901
  - ActiveSpaceTransformer extracted energy part: -246.703919646007
~ Nuclear repulsion energy (Hartree): 106.218168150552
> Total ground state energy (Hartree): -141.788858287356
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 1.000 S^2: 2.000 M: -1.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [-139.21936841  72.34588319  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [-141.72729882  74.38712216  -0.0081244]
    - computed part:      [-13.34150076  7.56574241  -0.00747228]
    - ActiveSpaceTransformer extracted energy part: [-128.38579806  66.82137974  -0.00065211]
  > Dipole moment (a.u.): [2.50793041  -2.04123897  0.0081244]  Total: 3.23364151
                 (debye): [6.37452266  -5.18831145  0.02065016]  Total: 8.21909613
 
Numpy Energy Result
=== GROUND STATE ENERGY ===
 
* El

In [15]:
# Record the end time
end_time = time.time()

# Calculate the duration
duration = end_time - start_time

# Print the duration
print("Job took {:.2f} seconds to run.".format(duration))

Job took 362.44 seconds to run.


In [16]:
start_time = time.time()

In [17]:
def get_qubit_op(
    molecule,
    basis,
    functional,
    method,
    driver_type= ElectronicStructureDriverType.PYSCF,
    mapper= JordanWignerMapper()):

  driver=ElectronicStructureMoleculeDriver(molecule, basis, method,driver_type)
  properties = driver.run()
  problem = ElectronicStructureProblem(driver)

  second_q_ops = problem.second_q_ops()
  second_q_ops = list(second_q_ops.values()) #list of second quantized operators
  hamiltonian=second_q_ops[0]

  numpy_solver = NumPyMinimumEigensolver()
  tmp = properties.get_property('ParticleNumber')
  alpha_occ=tmp.occupation_alpha
  beta_occ=tmp.occupation_beta

  mo_considered=3
  #active and inactive space has to be even, non-magnetic

  first_index = min(np.where(alpha_occ<1)[0][0],np.where(beta_occ<1)[0][0])

  orb_act = np.arange(first_index-mo_considered,first_index)+1

  transformer= ActiveSpaceTransformer(num_electrons=mo_considered+1, num_molecular_orbitals=len(orb_act), active_orbitals=orb_act)
  problem_reduced = ElectronicStructureProblem(driver, [transformer])
  second_q_ops_reduced = problem_reduced.second_q_ops()

  second_q_ops_reduced = list(second_q_ops_reduced.values())
  hamiltonian_reduced = second_q_ops_reduced[0]

  # mapper = JordanWignerMapper()
  converter = QubitConverter(mapper=mapper)
  qubit_op = converter.convert(hamiltonian_reduced)

  res1={}
  res1['qubit_op']=qubit_op
  res1['converter']=converter
  res1['problem_reduced']=problem_reduced
  res1['numpy_solver']=numpy_solver
  return res1 #qubit_op, converter, problem_reduced

def get_energy(optimizer='',device='',qubit_op='',seed='42', reps=1):
  #seed = 42
  counts = []
  values = []


  def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)

  algorithm_globals.random_seed = seed
  np.random.seed(seed)
  print('device',device)
  print('seed',seed)

  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
  n_qubits = qubit_op.num_qubits
  ansatz = QuantumCircuitLibrary(n_qubits=n_qubits, reps=reps).circuit6()
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)
  result = vqe.compute_minimum_eigenvalue(operator=qubit_op)


  eigenvalue = result.eigenvalue
  # return eigenvalue, vqe, qi

  res={}
  res['eigenvalue']=eigenvalue
  res['vqe']=vqe
  res['qi']=qi
  return res

elements=['Al']
basis=['631g','sto3g', '321g', 'sto6g', '631g','lanl2dz']
method=[MethodType.RKS]
#driver_type=[]
optimizer=[SLSQP(maxiter=1000)]
mapper=[JordanWignerMapper()]
functionals=['lda']
devices=[Aer.get_backend('statevector_simulator')]
reps_list = [1]
mem={}
for i in elements:
  for j in basis:
    for k in method:
      #for l in driver_type:
        for m in optimizer:
          for n in mapper:
            for o in functionals:
              for p in devices:
                for reps in reps_list:
                  print(i,j,k,m,n,o,p,reps)

                  res1=get_qubit_op(molecule=molecule,basis=j,mapper=n,functional=o,method=k)
                  res=get_energy(optimizer=m,device=p,qubit_op=res1['qubit_op'],seed=42)
                  print(res['eigenvalue'])

                  #GroundStateEigensolver
                  print("VQE Energy Result")
                  solver =GroundStateEigensolver(res1['converter'], res['vqe'])
                  result=solver.solve(res1['problem_reduced'])
                  print(result)

                  #numpy
                  print("Numpy Energy Result")
                  solver =GroundStateEigensolver(res1['converter'], res1['numpy_solver'])
                  result=solver.solve(res1['problem_reduced'])
                  print(result)

                  mem[i,j,k,m,n,o,p,reps]= {
                      'eigenvalue': res['eigenvalue'],
                      'vqe': res['vqe'],
                      'qi': res['qi'],
                      'converter': res1['converter'],
                      'problem_reduced': res1['problem_reduced']
                     }

Al 631g MethodType.RKS <qiskit.algorithms.optimizers.slsqp.SLSQP object at 0x7e9e542b5ba0> <qiskit_nature.mappers.second_quantization.jordan_wigner_mapper.JordanWignerMapper object at 0x7e9e542b4670> lda statevector_simulator 1


<ipython-input-17-eb8e343ad184>:17: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  numpy_solver = NumPyMinimumEigensolver()


device statevector_simulator
seed 42


<ipython-input-17-eb8e343ad184>:62: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
<ipython-input-17-eb8e343ad184>:65: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)


(8.5066017987e-08+0j)
VQE Energy Result
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -831.820030943739
  - computed part:      -1.371347552137
  - ActiveSpaceTransformer extracted energy part: -830.448683391601
~ Nuclear repulsion energy (Hartree): 106.218168150552
> Total ground state energy (Hartree): -725.601862793187
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.511 S^2: 0.771 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [-139.21936841  72.34588319  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [-142.79169612  74.14970555  -0.00023924]
    - computed part:      [-14.28151602  7.36856086  -0.00023924]
    - ActiveSpaceTransformer extracted energy part: [-128.5101801  66.78114469  0.0]
  > Dipole moment (a.u.): [3.57232771  -1.80382236  0.00023924]  Total: 4.0019121
                 (debye): [9.07995051  -4.58485868  0.00060809]  Total: 10.171845
 
Numpy Energy Result
=== GROUND STATE ENERGY ===
 
* Electronic g

...

(8.5066017987e-08+0j)
VQE Energy Result
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -831.820031094909
  - computed part:      -1.371347703308
  - ActiveSpaceTransformer extracted energy part: -830.448683391601
~ Nuclear repulsion energy (Hartree): 106.218168150552
> Total ground state energy (Hartree): -725.601862944358
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.511 S^2: 0.771 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [-139.21936841  72.34588319  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [-142.79125408  74.14982575  -0.00021478]
    - computed part:      [-14.28107398  7.36868105  -0.00021478]
    - ActiveSpaceTransformer extracted energy part: [-128.5101801  66.78114469  0.0]
  > Dipole moment (a.u.): [3.57188567  -1.80394256  0.00021478]  Total: 4.00157169
                 (debye): [9.07882693  -4.5851642  0.00054592]  Total: 10.17097977
 
Numpy Energy Result
=== GROUND STATE ENERGY ===
 
* Electronic

<ipython-input-17-eb8e343ad184>:17: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  numpy_solver = NumPyMinimumEigensolver()


device statevector_simulator
seed 42


<ipython-input-17-eb8e343ad184>:62: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
<ipython-input-17-eb8e343ad184>:65: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)


(8.5066017987e-08+0j)
VQE Energy Result
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -247.938526078923
  - computed part:      -1.239874935514
  - ActiveSpaceTransformer extracted energy part: -246.698651143409
~ Nuclear repulsion energy (Hartree): 106.218168150552
> Total ground state energy (Hartree): -141.720357928372
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.361 S^2: 0.492 M: -0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [-139.21936841  72.34588319  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [-137.42216007  77.78936866  0.00161682]
    - computed part:      [-9.34933279  10.88587387  -0.01251853]
    - ActiveSpaceTransformer extracted energy part: [-128.07282728  66.90349479  0.01413536]
  > Dipole moment (a.u.): [-1.79720834  -5.44348547  -0.00161682]  Total: 5.73249461
                 (debye): [-4.56804752  -13.83595868  -0.00410955]  Total: 14.57054658
 
Numpy Energy Result
=== GROUND STATE ENERGY ===


In [18]:
# Record the end time
end_time = time.time()

# Calculate the duration
duration = end_time - start_time

# Print the duration
print("Job took {:.2f} seconds to run.".format(duration))

Job took 361.44 seconds to run.


In [19]:
start_time = time.time()

In [20]:
def get_qubit_op(
    molecule,
    basis,
    functional,
    method,
    driver_type= ElectronicStructureDriverType.PYSCF,
    mapper= JordanWignerMapper()):

  driver=ElectronicStructureMoleculeDriver(molecule, basis, method,driver_type)
  properties = driver.run()
  problem = ElectronicStructureProblem(driver)

  second_q_ops = problem.second_q_ops()
  second_q_ops = list(second_q_ops.values()) #list of second quantized operators
  hamiltonian=second_q_ops[0]

  numpy_solver = NumPyMinimumEigensolver()
  tmp = properties.get_property('ParticleNumber')
  alpha_occ=tmp.occupation_alpha
  beta_occ=tmp.occupation_beta

  mo_considered=3
  #active and inactive space has to be even, non-magnetic

  first_index = min(np.where(alpha_occ<1)[0][0],np.where(beta_occ<1)[0][0])

  orb_act = np.arange(first_index-mo_considered,first_index)+1

  transformer= ActiveSpaceTransformer(num_electrons=mo_considered+1, num_molecular_orbitals=len(orb_act), active_orbitals=orb_act)
  problem_reduced = ElectronicStructureProblem(driver, [transformer])
  second_q_ops_reduced = problem_reduced.second_q_ops()

  second_q_ops_reduced = list(second_q_ops_reduced.values())
  hamiltonian_reduced = second_q_ops_reduced[0]

  # mapper = JordanWignerMapper()
  converter = QubitConverter(mapper=mapper)
  qubit_op = converter.convert(hamiltonian_reduced)

  res1={}
  res1['qubit_op']=qubit_op
  res1['converter']=converter
  res1['problem_reduced']=problem_reduced
  res1['numpy_solver']=numpy_solver
  return res1 #qubit_op, converter, problem_reduced

def get_energy(optimizer='',device='',qubit_op='',seed='42', reps=1):
  #seed = 42
  counts = []
  values = []


  def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)

  algorithm_globals.random_seed = seed
  np.random.seed(seed)
  print('device',device)
  print('seed',seed)

  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
  n_qubits = qubit_op.num_qubits
  ansatz = QuantumCircuitLibrary(n_qubits=n_qubits, reps=reps).circuit6()
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)
  result = vqe.compute_minimum_eigenvalue(operator=qubit_op)


  eigenvalue = result.eigenvalue
  # return eigenvalue, vqe, qi

  res={}
  res['eigenvalue']=eigenvalue
  res['vqe']=vqe
  res['qi']=qi
  return res

elements=['Al']
basis=['631g','sto3g', '321g', 'sto6g', '631g','lanl2dz']
method=[MethodType.RKS]
#driver_type=[]
optimizer=[SLSQP(maxiter=1000)]
mapper=[JordanWignerMapper()]
functionals=['lda']
devices=[Aer.get_backend('statevector_simulator')]
reps_list = [1]
mem={}
for i in elements:
  for j in basis:
    for k in method:
      #for l in driver_type:
        for m in optimizer:
          for n in mapper:
            for o in functionals:
              for p in devices:
                for reps in reps_list:
                  print(i,j,k,m,n,o,p,reps)

                  res1=get_qubit_op(molecule=molecule,basis=j,mapper=n,functional=o,method=k)
                  res=get_energy(optimizer=m,device=p,qubit_op=res1['qubit_op'],seed=42)
                  print(res['eigenvalue'])

                  #GroundStateEigensolver
                  print("VQE Energy Result")
                  solver =GroundStateEigensolver(res1['converter'], res['vqe'])
                  result=solver.solve(res1['problem_reduced'])
                  print(result)

                  #numpy
                  print("Numpy Energy Result")
                  solver =GroundStateEigensolver(res1['converter'], res1['numpy_solver'])
                  result=solver.solve(res1['problem_reduced'])
                  print(result)

                  mem[i,j,k,m,n,o,p,reps]= {
                      'eigenvalue': res['eigenvalue'],
                      'vqe': res['vqe'],
                      'qi': res['qi'],
                      'converter': res1['converter'],
                      'problem_reduced': res1['problem_reduced']
                     }

Al 631g MethodType.RKS <qiskit.algorithms.optimizers.slsqp.SLSQP object at 0x7e9e54265d80> <qiskit_nature.mappers.second_quantization.jordan_wigner_mapper.JordanWignerMapper object at 0x7e9e54265cf0> lda statevector_simulator 1


<ipython-input-20-eb8e343ad184>:17: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  numpy_solver = NumPyMinimumEigensolver()


device statevector_simulator
seed 42


<ipython-input-20-eb8e343ad184>:62: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
<ipython-input-20-eb8e343ad184>:65: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)


(8.5066017987e-08+0j)
VQE Energy Result
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -831.820031068331
  - computed part:      -1.371347676729
  - ActiveSpaceTransformer extracted energy part: -830.448683391601
~ Nuclear repulsion energy (Hartree): 106.218168150552
> Total ground state energy (Hartree): -725.601862917779
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.511 S^2: 0.771 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [-139.21936841  72.34588319  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [-142.79129886  74.14981068  -0.00022326]
    - computed part:      [-14.28111876  7.36866598  -0.00022326]
    - ActiveSpaceTransformer extracted energy part: [-128.5101801  66.78114469  0.0]
  > Dipole moment (a.u.): [3.57193045  -1.80392749  0.00022326]  Total: 4.00160487
                 (debye): [9.07894075  -4.58512589  0.00056747]  Total: 10.1710641
 
Numpy Energy Result
=== GROUND STATE ENERGY ===
 
* Electronic

...

(8.5066017987e-08+0j)
VQE Energy Result
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -831.820030900192
  - computed part:      -1.371347508591
  - ActiveSpaceTransformer extracted energy part: -830.448683391601
~ Nuclear repulsion energy (Hartree): 106.218168150552
> Total ground state energy (Hartree): -725.601862749641
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.511 S^2: 0.771 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [-139.21936841  72.34588319  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [-142.79172161  74.14969944  -0.00024758]
    - computed part:      [-14.2815415  7.36855475  -0.00024758]
    - ActiveSpaceTransformer extracted energy part: [-128.5101801  66.78114469  0.0]
  > Dipole moment (a.u.): [3.5723532  -1.80381625  0.00024758]  Total: 4.0019321
                 (debye): [9.08001527  -4.58484316  0.00062929]  Total: 10.17189582
 
Numpy Energy Result
=== GROUND STATE ENERGY ===
 
* Electronic g

<ipython-input-20-eb8e343ad184>:17: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.numpy_minimum_eigen_solver.NumPyMinimumEigensolver`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver``. See https://qisk.it/algo_migration for a migration guide.
  numpy_solver = NumPyMinimumEigensolver()


device statevector_simulator
seed 42


<ipython-input-20-eb8e343ad184>:62: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qi = QuantumInstance(device, seed_transpiler=seed, seed_simulator=seed)
<ipython-input-20-eb8e343ad184>:65: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi,callback=store_intermediate_result)


(8.5066017987e-08+0j)
VQE Energy Result
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -247.940096511668
  - computed part:      -1.240411107561
  - ActiveSpaceTransformer extracted energy part: -246.699685404106
~ Nuclear repulsion energy (Hartree): 106.218168150552
> Total ground state energy (Hartree): -141.721928361116
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.368 S^2: 0.503 M: -0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [-139.21936841  72.34588319  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [-137.44162274  77.68988472  -0.02868189]
    - computed part:      [-9.46321527  10.82141682  -0.02617615]
    - ActiveSpaceTransformer extracted energy part: [-127.97840747  66.8684679  -0.00250575]
  > Dipole moment (a.u.): [-1.77774567  -5.34400153  0.02868189]  Total: 5.6320116
                 (debye): [-4.51857836  -13.58309575  0.0729021]  Total: 14.31514426
 
Numpy Energy Result
=== GROUND STATE ENERGY ===
 
*

In [21]:
# Record the end time
end_time = time.time()

# Calculate the duration
duration = end_time - start_time

# Print the duration
print("Job took {:.2f} seconds to run.".format(duration))

Job took 371.80 seconds to run.
